In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
#Loading Dataset
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [ ]:
#Setup
x = tf.placeholder('float',[None,784])
y = tf.placeholder('float')

In [ ]:
class NeuralNetwork:
    def __init__(self,sizes):
        self.layers = []
        
        self.sess = tf.Session()
        
        for i in range(1,len(sizes)):
            self.layers.append(
               {'weights':tf.Variable(tf.random_normal([sizes[i-1], sizes[i]])),
                         'biases':tf.Variable(tf.random_normal([sizes[i]]))} 
            )
    
    def model(self,data):        
        l = tf.add(tf.matmul(data,self.layers[0]['weights']), self.layers[0]['biases'])
        l = tf.nn.relu(l)
        
        for i in range(1,len(self.layers)-1):
            l = tf.add(tf.matmul(l,self.layers[i]['weights']), self.layers[i]['biases'])
            l = tf.nn.relu(l)
    
        l = tf.matmul(l,self.layers[-1]['weights']) + self.layers[-1]['biases']
        
        return l
    
    def train(self,x,hm_epochs=10,batch_size=100):
        prediction = self.model(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction,y))

        optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

        self.sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0

            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _, c = self.sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch',epoch+1, 'completed out of',hm_epochs,'loss:', epoch_loss)

        self.prediction = prediction
        
            
    def score (self,features,labels):
        correct = tf.equal(tf.argmax(self.prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: ',accuracy.eval({x:features, y:labels},self.sess))

In [ ]:
layers = [784,100,10]
nn = NeuralNetwork(layers)
nn.train(x)
nn.score(mnist.test.images, mnist.test.labels)